In [1]:
import pandas as pd
from google.cloud import storage
from google.oauth2 import service_account
import requests
from pathlib import WindowsPath
import subprocess
import os

In [19]:
cloud_token = {
  "access_token": "ya29.a0Ad52N380j0ZKqAyX93CjoNuQ6oE3RgoeJWcASBAR2guKpRisiQGn8Ai2OL7Ha_S82Rsm3OYuvU9DVLexi-eZo7nRQyH243Rw_Ek6nC7z4T9TatV_e4T83nfQ6Yh_hKflNm3S_HS02uH-g4bPhCI28AFGG6mTRKWQhwuDaCgYKAbISARISFQHGX2Mim-yYzlThX6Aouji_xbQ1qQ0171", 
  "scope": "https://www.googleapis.com/auth/devstorage.full_control https://www.googleapis.com/auth/devstorage.read_only https://www.googleapis.com/auth/devstorage.read_write", 
  "token_type": "Bearer", 
  "expires_in": 3599, 
  "refresh_token": "1//04UrF7OBQa-ZOCgYIARAAGAQSNwF-L9IrO90-7YzoyHnjNTLWx_r9GSuyhJKioF58TInnYdWgDQDaZZMzV6uqV_TuCyHR8whVbkQ"
}

In [4]:
df = pd.read_csv("data/pifsc_products_detections_annotations.csv")
df

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00
...,...,...,...,...,...,...,...,...,...,...
38852,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,910,False,True,Background,0.000,75.000,2016-04-27 10:00:00+00:00,2016-04-27 10:01:15+00:00
38853,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1018,False,True,Background,0.000,75.000,2016-05-03 13:31:15+00:00,2016-05-03 13:32:30+00:00
38854,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,137,False,True,Background,0.000,75.000,2016-05-04 13:31:15+00:00,2016-05-04 13:32:30+00:00
38855,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,430,False,True,Background,0.000,75.000,2016-05-13 01:30:00+00:00,2016-05-13 01:31:15+00:00


In [9]:
df['label'].value_counts()

label
Mn            32189
Background     5928
Other           306
Vessel          242
Fish             98
Device           94
Name: count, dtype: int64

In [30]:
df[df['flac_compressed_xwav_object'] == "gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac"]

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00


In [4]:
df['audit_name'].unique()

array(['initial', 'model1', 'model2', 'model3', 'model4', 'postpub',
       'segments', 'testexpand', 'validation'], dtype=object)

In [12]:
df[df['label_is_strong'] == True].drop_duplicates()['label'].value_counts()

label
Mn       29979
Other      280
Name: count, dtype: int64

In [13]:
df[df['label_is_strong'] == False].drop_duplicates()['label'].value_counts()

label
Background    5700
Mn            1194
Vessel         230
Fish            97
Device          94
Name: count, dtype: int64

In [ ]:
df[df['label_is_strong']]

In [32]:
df['flac_compressed_xwav_object'].nunique()

5489

In [17]:
gcf = "gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac"

In [23]:
storage_credentials = service_account.Credentials.from_service_account_info(cloud_token)
# storage_clint = storage.Client()

MalformedError: Service account info was not in the expected format, missing fields token_uri, client_email.

In [33]:
bucket_name = "noaa-passive-bioacoustic"
blob_name = "pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac"
url = f"https://storage.googleapis.com/storage/v1/b/{bucket_name}/o/{blob_name}?alt=media"

headers = {"Authorization": f"Bearer {cloud_token['access_token']}"}

r = requests.get("https://storage.googleapis.com/noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac",headers=headers)

In [29]:
with open("data/audio_data/test_flac.flac", "wb") as f:
    f.write(r.content)

In [73]:
df[df['flac_compressed_xwav_object'] == "gs://noaa-passive-bioacoustic/pifsc/audio/pipan/hawaii/pipan_hawaii_02/audio/Hawaii_K_02_080506_183545.d20.x.flac"]

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
36,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,0,True,True,Mn,49.093,50.126,2008-05-06 18:36:34.093000+00:00,2008-05-06 18:36:35.126000+00:00
37,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,0,True,True,Mn,52.789,53.350,2008-05-06 18:36:37.789000+00:00,2008-05-06 18:36:38.350000+00:00
38,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,0,True,True,Mn,55.521,56.078,2008-05-06 18:36:40.521000+00:00,2008-05-06 18:36:41.078000+00:00
39,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,4,True,True,Mn,10.539,11.205,2008-05-06 18:43:55.539000+00:00,2008-05-06 18:43:56.205000+00:00
40,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,4,True,True,Mn,15.344,15.852,2008-05-06 18:44:00.344000+00:00,2008-05-06 18:44:00.852000+00:00
41,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,4,True,True,Mn,54.471,54.856,2008-05-06 18:44:39.471000+00:00,2008-05-06 18:44:39.856000+00:00
42,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Mn,1.290,2.902,2008-05-06 18:48:01.290000+00:00,2008-05-06 18:48:02.902000+00:00
43,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Mn,17.341,18.637,2008-05-06 18:48:17.341000+00:00,2008-05-06 18:48:18.637000+00:00
44,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Mn,22.018,23.104,2008-05-06 18:48:22.018000+00:00,2008-05-06 18:48:23.104000+00:00
45,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Mn,38.209,39.225,2008-05-06 18:48:38.209000+00:00,2008-05-06 18:48:39.225000+00:00


In [72]:
df[df['label_is_strong']==True][:100]

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00
...,...,...,...,...,...,...,...,...,...,...
95,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,18,True,True,Mn,57.142,57.774,2009-05-05 17:48:27.142000+00:00,2009-05-05 17:48:27.774000+00:00
96,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,18,True,True,Mn,60.996,61.839,2009-05-05 17:48:30.996000+00:00,2009-05-05 17:48:31.839000+00:00
97,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,18,True,True,Mn,65.178,66.302,2009-05-05 17:48:35.178000+00:00,2009-05-05 17:48:36.302000+00:00
98,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,18,True,True,Mn,68.481,69.606,2009-05-05 17:48:38.481000+00:00,2009-05-05 17:48:39.606000+00:00


In [45]:
files = list(df[df['label_is_strong']==True][:100]['flac_compressed_xwav_object'].unique())
files = [file[5:] for file in files]
files

['noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050702_031345.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050801_074615.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050801_110500.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050908_025000.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120313_165615.df20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120419_012845.df20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/hawaii/pipan_hawaii_01/audio/Hawaii_K_01_070813_065730.d20.x.flac',
 'noaa-passive-bioacoustic/pifsc/audio/pipan/hawaii/pipan_hawaii_01/audio/Hawaii_K_

In [48]:
# Download data
for file in files:
    url = f"https://storage.googleapis.com/{file}"
    r = requests.get(url,headers=headers)
    if r.status_code != 200:
        print(f"Failed to download {file}")
        break
    with open(f"data/audio_data/downloads/{file.split('/')[-1]}", "wb") as f:
        f.write(r.content)

['Cross_A_01_050606_123845.d20.x.flac',
 'Cross_A_01_050702_031345.d20.x.flac',
 'Cross_A_01_050801_074615.d20.x.flac',
 'Cross_A_01_050801_110500.d20.x.flac',
 'Cross_A_01_050908_025000.d20.x.flac',
 'Equator_A_01_120313_165615.df20.x.flac',
 'Equator_A_01_120419_012845.df20.x.flac',
 'Hawaii_K_01_070813_065730.d20.x.flac',
 'Hawaii_K_01_070813_073500.d20.x.flac',
 'Hawaii_K_01_070915_204730.d20.x.flac',
 'Hawaii_K_01_070915_212500.d20.x.flac',
 'Hawaii_K_02_080506_173715.d20.x.flac',
 'Hawaii_K_02_080506_183545.d20.x.flac',
 'Hawaii_K_02_080609_205945.d20.x.flac',
 'Hawaii_K_02_080702_115545.d20.x.flac',
 'Hawaii_K_02_080702_125715.d20.x.flac',
 'Hawaii_K_03_080807_141730.d20.x.flac',
 'Hawaii_K_03_080807_161500.d20.x.flac',
 'Hawaii_K_03_080815_114500.d20.x.flac',
 'Hawaii_K_03_080921_234730.d20.x.flac',
 'Hawaii_K_03_080922_014500.d20.x.flac',
 'Hawaii_K_03_081010_164730.d20.x.flac',
 'Hawaii_K_06_090505_164500.d20.x.flac']

In [108]:
# Uncompress data
files = os.listdir("data/audio_data/downloads")
for file in files:
    subprocess.run(f"C:/Windows/flac.exe -df --delete-input-file --preserve-modtime --keep-foreign-metadata ./data/audio_data/uncompressed/{file}", shell=True, capture_output=True)